In [14]:
import openpathsampling as paths
storage = paths.storage.Storage("mistis.nc", "r")

In [15]:
mistis = storage.networks.load(0)

In [18]:
# this should be true if we restored correctly
for ens in mistis.sampling_transitions[0].ensembles:
    print ens in storage.ensembles[:], repr(ens)


True <openpathsampling.ensemble.TISEnsemble object at 0x117178850>
True <openpathsampling.ensemble.TISEnsemble object at 0x11715c8d0>
True <openpathsampling.ensemble.TISEnsemble object at 0x117120d50>


In [20]:
mistis.hist_args['max_lambda'] = { 'bin_width' : 0.02, 'bin_range' : (-0.3, 0.5) }
mistis.hist_args['pathlength'] = { 'bin_width' : 5, 'bin_range' : (0, 150) }

In [21]:
movers = paths.MoveScheme(mistis)
movers.move_summary(storage, 'shooting')

OneWayShootingMover I'face 3 ran 3.65448504983% of the cycles with acceptance 6/11 (0.545454545455) 
OneWayShootingMover I'face 1 ran 4.98338870432% of the cycles with acceptance 10/15 (0.666666666667) 
OneWayShootingMover I'face 1 ran 5.31561461794% of the cycles with acceptance 10/16 (0.625) 
OneWayShootingMover I'face 0 ran 3.32225913621% of the cycles with acceptance 7/10 (0.7) 
OneWayShootingMover I'face 2 ran 3.65448504983% of the cycles with acceptance 5/11 (0.454545454545) 
OneWayShootingMover I'face 1 ran 4.98338870432% of the cycles with acceptance 8/15 (0.533333333333) 
OneWayShootingMover I'face 0 ran 5.98006644518% of the cycles with acceptance 17/18 (0.944444444444) 
OneWayShootingMover I'face 2 ran 6.3122923588% of the cycles with acceptance 15/19 (0.789473684211) 
OneWayShootingMover I'face 2 ran 3.98671096346% of the cycles with acceptance 8/12 (0.666666666667) 
OneWayShootingMover I'face 0 ran 5.31561461794% of the cycles with acceptance 15/16 (0.9375) 


In [22]:
mistis.rate_matrix(storage)

,"({x|opX(x) in [0.3, inf]} and {x|opY(x) in [-inf, -0.3]})","({x|opX(x) in [-inf, -0.3]} and {x|opY(x) in [0.3, inf]})","({x|opX(x) in [-inf, -0.3]} and {x|opY(x) in [-inf, -0.3]})"
"({x|opX(x) in [0.3, inf]} and {x|opY(x) in [-inf, -0.3]})",NaN,NaN,NaN
"({x|opX(x) in [-inf, -0.3]} and {x|opY(x) in [-inf, -0.3]})",NaN,NaN,NaN


In [23]:
trans = mistis.transitions.values()[0]
for ens in trans.ensembles:
    for step in storage.steps:
        assert(ens in [s.ensemble for s in step.active])